# Filtrado de proteínas virales objetivo
Se filtró las 10 poliproteínas virales objetivo para utilizar solamente las que presenten
antigenicidad y sean estables. Además, se evaluó la presencia de péptidos señal, hélices transmembrana y la localización subcelular de las proteínas filtradas.

In [5]:
# Instalar el paquete 'selenium'
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Importar los módulos necesarios de 'selenium'
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select

## 1. Predicción de antigenicidad

In [3]:
# Establecer la ubicación del WebDriver
s = Service('/usr/local/bin/chromedriver')

La predicción de la antigenicidad de las 10 secuencias proteicas se realizó por raspado web del servidor en línea para el análisis de antigenicidad, **Vaxijen v2.0** (http://www.ddg-pharmfac.net/vaxijen/VaxiJen/VaxiJen.html) (*Doytchinova & Flower, 2007*), manteniendo el <u>umbral en 0.4</u> y seleccionando a <u>'Virus'</u> como organismo objetivo. 

In [9]:
# Inicializar el Selenium WebDriver
driver = webdriver.Chrome(service = s)

# Visitar al servidor en línea VaxiJen v2.0
driver.get('http://www.ddg-pharmfac.net/vaxijen/VaxiJen/VaxiJen.html')

# Editar los parámetros
## Cargar el archivo input
select_file = driver.find_element(by = By.NAME, value = 'uploaded_file')
select_file.send_keys('/home/victor/Escritorio/Tesis/RESULTADOS/3. Filtrado de proteínas virales objetivo/Uniprot_Proteínas_virales_DENV-2')

## Seleccionar el organismo objetivo
select_target = Select(driver.find_element(by = By.NAME, value = 'Target'))
select_target.select_by_visible_text('Virus')

## Definir el valor umbral
threshold = driver.find_element(by = By.NAME, value = 'threshold')
threshold.send_keys('0.4')


# Entregar la data
driver.find_element(by = By.NAME, value = 'submit').click()


# Obtener la predicción antigénica 
get_url = driver.current_url
name_input = driver.find_element(By.XPATH, '//table/tbody/tr[4]/td[3]/table/tbody/tr/td').text

## Descargar y visualizar el output obtenido
resultados_VaxiJen = '/home/victor/Escritorio/Tesis/RESULTADOS/3. Filtrado de proteínas virales objetivo/VaxiJen_v2_Resultados/resultados_antigenicidad'
with open(resultados_VaxiJen, 'w') as file:
    print(name_input, file = file)

with open('/home/victor/Escritorio/Tesis/RESULTADOS/3. Filtrado de proteínas virales objetivo/VaxiJen_v2_Resultados/resultados_antigenicidad') as f:
    antigenicidad = f.read()
    print(antigenicidad)
    
driver.close()

Model selected: virus
Threshold for this model: 0.4



Your Sequence:

>ancC_ProteÃ­na de la cÃ¡pside anclada

MNNQRKKARNTPFNMLKRERNRVSTVQQLTK
RFSLGMLQGRGPLKLFMALVAFLRFLTIPPT
AGILKRWGTIKKSKAINVLRGFRKEIGRMLN
ILNRRRR


Overall Prediction for the Protective Antigen = 0.3693 ( Probable NON-ANTIGEN ).


Your Sequence:

>prM_ProteÃ­na premembrana

FHLTTRNGEPHMIVSRQEKGKSLLFKTKDGT
NMCTLMAMDLGELCEDTITYKCPFLKQNEPE
DIDCWCNSTSTWVTYGTCTTTGEHRREKRSV
ALVPHVGMGLETRTETWMSSEGAWKHAQRIE
TWILRHPGFTIMAAILAYTIGTTHFQRVLIF
ILLTAIAPSMT


Overall Prediction for the Protective Antigen = 0.6442 ( Probable ANTIGEN ).


Your Sequence:

>E_ProteÃ­na de envoltura

MRCIGISNRDFVEGVSGGSWVDIVLEHGSCV
TTMAKNKPTLDFELIKTEAKQPATLRKYCIE
AKLTNTTTDSRCPTQGEPTLNEEQDKRFVCK
HSMVDRGWGNGCGLFGKGGIVTCAMFTCKKN
MEGKIVQPENLEYTVVITPHSGEEHAVGNDT
GKHGKEVKITPQSSITEAELTGYGTVTMECS
PRTGLDFNEMVLLQMEDKAWLVHRQWFLDLP
LPWLPGADTQGSNWIQKETLVTFKNPHAKKQ
DVVVLGSQEGAMHTALTGATEIQMSSGNLLF
TGHLKCRLRMDKLQLKGMSYSMCTGKFKIVK
EIAETQHGTIVIRVQYEGDGSPCKIPFEIMD
LEKRHVLG

De igual manera, los resultados obtenidos se compararon con los predichos por la herramienta **ANTIGENpro**, integrada en el conjunto de predictores **SCRATCH** (*Cheng et al., 2005*) (https://scratch.proteomics.ics.uci.edu/). Las proteínas predichas como <u>'No antigénico'</u> por VaxiJen serán descartadas para el resto de la metodología. 

In [47]:
# Diseñar un archivo de entrada exclusivamente con los nombres de las proteínas
file_name = open('Uniprot_Proteínas_virales_DENV-2')

## Nombres de cada proteína
data_name = []
for pos_name, l_num_name in enumerate(file_name):
    # comprobar si el número de línea se especifica en las líneas para leer la matriz
    if pos_name %3 == 0:
        # imprimir el número de línea requerido
        data_name.append(l_num_name)
        
each_name = [[y.rstrip('\n')] for y in data_name]

In [42]:
# Diseñar un archivo de entrada exclusivamente con las secuencias de las proteínas
file_seq = open('Uniprot_Proteínas_virales_DENV-2')

## Secuencias aminoacídicas de cada proteína
data_seq = []
for pos, l_num in enumerate(file_seq):
    # comprobar si el número de línea se especifica en las líneas para leer la matriz
    if pos %3 == 1:
        # imprimir el número de línea requerido
        data_seq.append(l_num)

each_seq = [[x.rstrip('\n')] for x in data_seq]

In [48]:
# Unir los dos tipos de archivos de entrada en una sola lista
def countList(lst1, lst2):
    return [sub[item] for item in range(len(lst2))
                      for sub in [lst1, lst2]]
    
all_proteins = countList(data_name, data_seq)

all_proteins_wn = list(map(lambda x:x.strip(),all_proteins))

In [49]:
# Separar los elementos por pares (cada lista tendrá un nombre y una secuencia)
final_list = list()
chunk_size = 2

for i in range(0, len(all_proteins_wn), chunk_size):
    final_list.append(all_proteins_wn[i:i+chunk_size])

In [40]:
# Eliminar los [] externos y guardar el formato final de la lista a utilizar en ANTIGENpro 
final_list = ['>ancC_Proteína de la cápside anclada', 'MNNQRKKARNTPFNMLKRERNRVSTVQQLTKRFSLGMLQGRGPLKLFMALVAFLRFLTIPPTAGILKRWGTIKKSKAINVLRGFRKEIGRMLNILNRRRR'], ['>prM_Proteína premembrana', 'FHLTTRNGEPHMIVSRQEKGKSLLFKTKDGTNMCTLMAMDLGELCEDTITYKCPFLKQNEPEDIDCWCNSTSTWVTYGTCTTTGEHRREKRSVALVPHVGMGLETRTETWMSSEGAWKHAQRIETWILRHPGFTIMAAILAYTIGTTHFQRVLIFILLTAIAPSMT'], ['>E_Proteína de envoltura', 'MRCIGISNRDFVEGVSGGSWVDIVLEHGSCVTTMAKNKPTLDFELIKTEAKQPATLRKYCIEAKLTNTTTDSRCPTQGEPTLNEEQDKRFVCKHSMVDRGWGNGCGLFGKGGIVTCAMFTCKKNMEGKIVQPENLEYTVVITPHSGEEHAVGNDTGKHGKEVKITPQSSITEAELTGYGTVTMECSPRTGLDFNEMVLLQMEDKAWLVHRQWFLDLPLPWLPGADTQGSNWIQKETLVTFKNPHAKKQDVVVLGSQEGAMHTALTGATEIQMSSGNLLFTGHLKCRLRMDKLQLKGMSYSMCTGKFKIVKEIAETQHGTIVIRVQYEGDGSPCKIPFEIMDLEKRHVLGRLITVNPIVTEKDSPVNIEAEPPFGDSYIIIGAEPGQLKLDWFKKGSSIGQMFETTMRGAKRMAILGDTAWDFGSLGGVFTSIGKALHQVFGAIYGAAFSGVSWTMKILIGVIITWIGMNSRSTSLSVSLVLVGIVTLYLGVMVQA'], ['>NS1_Proteína no estructural 1', 'DSGCVVSWKNKELKCGSGIFVTDNVHTWTEQYKFQPESPSKLASAIQKAHEEGICGIRSVTRLENLMWKQITSELNHILSENEVKLTIMTGDIKGIMQVGKRSLRPQPTELRYSWKTWGKAKMLSTELHNQTFLIDGPETAECPNTNRAWNSLEVEDYGFGVFTTNIWLRLREKQDAFCDSKLMSAAIKDNRAVHADMGYWIESALNDTWKIEKASFIEVKSCHWPKSHTLWSNGVLESEMVIPKNIAGPVSQHNNRPGYHTQTAGPWHLGKLEMDFDFCEGTTVVVTEECGNRGPSLRTTTASGKLITEWCCRSCTLPPLRYRGEDGCWYGMEIRPLKEKEENLVSSLVTA'], ['>NS2A_Proteína no estructural 2A', 'GHGQIDNFSLGILGMALFLEEMLRTRVGTKHAILLVAVSFLTLITGNMSFRDLGRVMVMVGATMTDDIGMGVTYLALLAAFKVRPTFAAGLLLRKLTSKELMMTTIGIVLLSQSSIPETILELTDALALGMMVLKMVRNMEKYQLAVTIMAILCVPNAVILQNAWKVSCTTLAVVSVSPLLLTSSQQKADWIPLALTIKGLNPTAIFLTTLTRTSKKR'], ['>NS2B_Proteína no estructural 2B', 'SWPLNEAIMAVGMVSILASSLLKNDIPMTGPLVAGGLLTVCYVLTGRSADLELERATDVKWDDQAEISGSSPILSITISEDGSMSIKNEEEEQTLTILIRTGLLVISGLFPVSIPITAAAWYLWEVKKQR'], ['>NS3_Proteína no estructural 3', 'AGVLWDVPSPPPVGRAELEDGAYRIKQKGILGYSQIGAGVYKEGTFHTMWHVTRGAVLMHKGKRIEPSWADVKKDLISYGGGWKLEGEWKEGEEVQVLALEPGKNPRAVQTKPGLFRTNTGTIGAVSLDFSPGTSGSPIVDKKGKVVGLYGNGVVTRGGAYVSAIAQTEKGIEDNPEIEDDIFRKRRLTIMDLHPGAGKTKRYLPAIVREAIKRGLRTLILAPTRVVAAEMEEALRGLPIRYQTPAIRAEHTGREIVDLMCHATFTMRLLSPIRVPNYNLIIMDEAHFTDPASIAARGYISTRVEMGEAAGIFMTATPPGSRDPFPQSNAPIMDEEREIPERSWNSGHEWVTDFKGKTVWFVPSIKTGNDIAACLRKNGKRVIQLSRKTFDSEYVKTRTNDWDFVVTTDISEMGANFKAERVIDPRRCMKPVILTDGEERVILAGPMPVTHSSAAQRRGRIGRNPRNENDQYIYMGEPLENDEDCAHWKEAKMLLDNINTPEGIIPSMFEPEREKVDAIDGEYRLRGEARKTFVDLMRRGDLPVWLAYKVAAEGINYADRRWCFDGTRNNQILEENVEVEIWTKEGERKKLKPRWLDARIYSDPLALKEFKEFAAGRK'], ['>NS4A_Proteína no estructural 4A', 'SLTLNLITEMGRLPTFMTQKARDALDNLAVLHTAEAGGKAYNHALSELPETLETLLLLTLLATVTGGIFLFLMSGRGIGKMTLGMCCIITASILLWYAQIQPHWIAASIILEFFLIVLLIPEPEKQR'], ['>NS4B_Proteína no estructural 4B', 'NEMGFLEKTKKDLGLGHIATQQPESNILDIDLRPASAWTLYAVATTFITPMLRHSIENSSVNVSLTAIANQATVLMGLGKGWPLSKMDIGVPLLAIGCYSQVNPITLTAALLMLVAHYAIIGPGLQAKATREAQKRAAAGIMKNPTVDGITVIDLDPIPYDPKFEKQLGQVMLLVLCVTQVLMMRTTWALCEALTLATGPVSTLWEGNPGRFWNTTIAVSMANIFRGSYLAGAGLLFSIMKNTTSTRR'], ['>NS5_Proteína no estructural 5', 'GTGNMGETLGEKWKNRLNALGKSEFQIYKKSGIQEVDRTLAKEGIKRGETDHHAVSRGSAKLRWFVERNLVTPEGKVVDLGCGRGGWSYYCGGLKNVREVKGLTKGGPGHEEPIPMSTYGWNLVRLQSGVDVFFVPPEKCDTLLCDIGESSPNPTVEAGRTLRVLNLVENWLNNNTQFCVKVLNPYMPSVIERMETLQRKYGGALVRNPLSRNSTHEMYWVSNASGNIVSSVNMISRMLINRFTMRHKKATYEPDVDLGSGTRNIGIESETPNLDIIGKRIEKIKQEHETSWHYDQDHPYKTWAYHGSYETKQTGSASSMVNGVVRLLTKPWDVIPMVTQMAMTDTTPFGQQRVFKEKVDTRTQEPKEGTKKLMKITAEWLWKELGKKKTPRMCTREEFTKKVRSNAALGAIFTDENKWKSAREAVEDNRFWELVDKERNLHLEGKCETCVYNMMGKREKKLGEFGKAKGSRAIWYMWLGARFLEFEALGFLNEDHWFSRENSLSGVEGEGLHKLGYILREVSKKEGGAMYADDTAGWDTRITIEDLKNEEMITNHMAGEHKKLAEAIFKLTYQNKVVRVQRPTPRGTVMDIISRRDQRGSGQVVTYGLNTFTNMEAQLIRQMEGEGVFKSIQHLTASEEIAVQDWLVRVGRERLSRMAISGDDCVVKPLDDRFAKALTALNDMGKVRKDIQQWEPSRGWNDWTQVPFCSHHFHELIMKDGRTLVVPCRNQDELIGRARISQGAGWSLRETACLGKSYAQMWSLMYFHRRDLRLAANAICSAVPSHWVPTSRTTWSIHASHEWMTTEDMLTVWNRVWILENPWMEDKTPVESWEEIPYLGKREDQWCGSLIGLTSRATWAKNIQTAINQVRSLIGNEEYTDYMPSMKRFRREEEEVGVLW']

In [196]:
# Crear una lista con las listas de las primeras 5 secuencias con sus nombres respectivos
first_5_sequences = end[0:5]

In [5]:
# Importar el módulo 'time'
import time

In [197]:
# OBTENER LOS RESULTADOS DE ANTIGENICIDAD CON ANTIGENpro PARA LAS 1ERAS 5 SECUENCIAS 
# (El servidor solo acepta un máximo de 5 secuencias enviadas hasta que se envíen sus resultados por correo)

# Inicializar el Selenium WebDriver
driver = webdriver.Chrome(service = s)

# Definir los parámetros:
## Website de la herramienta ANTIGENpro en SCRATCH
ANTIGENpro = driver.get('https://scratch.proteomics.ics.uci.edu/')

### Esperar un segundo, hasta que la página se cargue por completo
time.sleep(1)

## E-mail de destino
email = driver.find_element(by = By.NAME, value = 'email')

## Nombre de la proteína objetivo 
protein = driver.find_elements(by = By.XPATH, value = '/html/body/div/form/table/tbody/tr[2]/td[2]/input')

## Secuencia de la proteína objetivo
sequence = driver.find_elements(by = By.XPATH, value = '/html/body/div/form/table/tbody/tr[3]/td[2]/textarea')

## Selección de la opción 'ANTIGENpro: Protein Antigenicity'
submitButton = driver.find_element(by = By.NAME, value = 'ant')

## Entrega de la data
send_response = driver.find_element(by = By.XPATH, value = '/html/body/div[1]/form/table/tbody/tr[6]/td/input[2]')


# Realizar el análisis de antigenicidad:
for data in first_5_sequences:
    count = 0
    for value in protein:
        value.send_keys(data[count])
        count += 1
        
    for value in sequence:
        value.send_keys(data[count])
        count += 1 
        
        email.send_keys("victor.cornejo@unmsm.edu.pe")
        submitButton.click()
        send_response.click()
        driver.find_element(by = By.LINK_TEXT, value = 'Back to Home').click()
        
        # Volver a definir los elementos web debido al cambio de DOM tras la actualización
        email = driver.find_element(by = By.NAME, value = 'email')
        protein = driver.find_elements(by = By.XPATH, value = '/html/body/div/form/table/tbody/tr[2]/td[2]/input')
        sequence = driver.find_elements(by = By.XPATH, value = '/html/body/div/form/table/tbody/tr[3]/td[2]/textarea')
        submitButton = driver.find_element(by = By.NAME, value = 'ant')
        send_response = driver.find_element(by = By.XPATH, value = '/html/body/div[1]/form/table/tbody/tr[6]/td/input[2]')

driver.close()

print("The first 5 sequences have been sent successfully")

The first 5 sequences have been sent successfully


In [200]:
# Crear una lista con las listas de las últimas 5 secuencias con sus nombres respectivos
last_5_sequences = end[5:10]

In [201]:
# OBTENER LOS RESULTADOS DE ANTIGENICIDAD CON ANTIGENpro PARA LAS ÚLTIMAS 5 SECUENCIAS
# (Utilizar el mismo correo luego de 2 min aprox o emplear otro)

# Inicializar el Selenium WebDriver
driver = webdriver.Chrome(service = s)

# Definir los parámetros:
## Website de la herramienta ANTIGENpro en SCRATCH
ANTIGENpro = driver.get('https://scratch.proteomics.ics.uci.edu/')

# Esperar un segundo, hasta que la página se cargue por completo
time.sleep(1)

## E-mail de destino
email = driver.find_element(by = By.NAME, value = 'email')

## Nombre de la proteína objetivo 
protein = driver.find_elements(by = By.XPATH, value = '/html/body/div/form/table/tbody/tr[2]/td[2]/input')

## Secuencia de la proteína objetivo
sequence = driver.find_elements(by = By.XPATH, value = '/html/body/div/form/table/tbody/tr[3]/td[2]/textarea')

## Selección de la opción 'ANTIGENpro: Protein Antigenicity'
submitButton = driver.find_element(by = By.NAME, value = 'ant')

## Entrega de la data
send_response = driver.find_element(by = By.XPATH, value = '/html/body/div[1]/form/table/tbody/tr[6]/td/input[2]')


# Realizar el análisis de antigenicidad:
for data in last_5_sequences:
    count = 0
    for value in protein:
        value.send_keys(data[count])
        count += 1
        
    for value in sequence:
        value.send_keys(data[count])
        count += 1 
        
        email.send_keys("victor.cornejo@unmsm.edu.pe")
        submitButton.click()
        send_response.click()
        driver.find_element(by = By.LINK_TEXT, value = 'Back to Home').click()
        
        email = driver.find_element(by = By.NAME, value = 'email')
        protein = driver.find_elements(by = By.XPATH, value = '/html/body/div/form/table/tbody/tr[2]/td[2]/input')
        sequence = driver.find_elements(by = By.XPATH, value = '/html/body/div/form/table/tbody/tr[3]/td[2]/textarea')
        submitButton = driver.find_element(by = By.NAME, value = 'ant')
        send_response = driver.find_element(by = By.XPATH, value = '/html/body/div[1]/form/table/tbody/tr[6]/td/input[2]')

driver.close()

print("The last 5 sequences have been sent successfully")

The last 5 sequences have been sent successfully


## 2. Análisis de propiedades fisicoquímicas

Las propiedades fisicoquímicas de las secuencias se determinaron mediante un script en Python del módulo **Bio.SeqUtils.ProtParam**, basado en la herramienta en línea **ProtParam** (https://web.expasy.org/protparam/) de ExPASy (*Wilkins et al., 1999*). Las proteínas predichas como <u>'Inestable'</u> fueron descartadas para el resto de la metodología.

In [3]:
# Instalar los paquetes 'biopython' y 'peptides'
pip install biopython
pip install peptides

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.4/115.4 KB 1.4 MB/s eta 0:00:000:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [68]:
# Importar los módulos 'Bio', 'collections' y 'peptides'
import Bio
import collections
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import peptides

In [15]:
# Determinar cada una una de las propiedades fisicoquímicas de las proteínas:
## Número de aminoácidos - len()
for rec in SeqIO.parse('Filtro_Proteínas_virales_DENV-2.fasta', 'fasta'):
    total_amino_acid = len(rec.seq)
    print(rec.id, total_amino_acid)

prM_Proteína 166
E_Proteína 495
NS1_Proteína 352
NS2A_Proteína 218
NS2B_Proteína 130
NS3_Proteína 618
NS4A_Proteína 127
NS4B_Proteína 248
NS5_Proteína 900


In [192]:
## Peso molecular - molecular_weight()
for rec in SeqIO.parse('Filtro_Proteínas_virales_DENV-2.fasta', 'fasta'):
    x = ProteinAnalysis(rec.seq)
    print(rec.id, '%0.2f' % x.molecular_weight())

prM_Proteína 18871.60
E_Proteína 54324.24
NS1_Proteína 39893.92
NS2A_Proteína 23656.31
NS2B_Proteína 14061.12
NS3_Proteína 69489.57
NS4A_Proteína 13966.51
NS4B_Proteína 26774.13
NS5_Proteína 103269.54


In [193]:
## Punto isoeléctrico - isoelectric_point()
for rec in SeqIO.parse('Filtro_Proteínas_virales_DENV-2.fasta', 'fasta'):
    x = ProteinAnalysis(rec.seq)
    print(rec.id, '%0.2f' % x.isoelectric_point())

prM_Proteína 7.19
E_Proteína 6.86
NS1_Proteína 6.41
NS2A_Proteína 9.82
NS2B_Proteína 4.43
NS3_Proteína 8.53
NS4A_Proteína 5.84
NS4B_Proteína 9.01
NS5_Proteína 8.77


In [195]:
## Coeficiente de extinción - molar_extinction_coefficient()
for rec in SeqIO.parse('Filtro_Proteínas_virales_DENV-2.fasta', 'fasta'):
    x = ProteinAnalysis(rec.seq)
    print(rec.id, x.molar_extinction_coefficient())

prM_Proteína (31970, 32345)
E_Proteína (66920, 67670)
NS1_Proteína (87430, 88180)
NS2A_Proteína (13980, 14105)
NS2B_Proteína (24980, 24980)
NS3_Proteína (102330, 102580)
NS4A_Proteína (13980, 14105)
NS4B_Proteína (34950, 35075)
NS5_Proteína (215770, 216645)


In [196]:
## Índice de inestabilidad - instability_index()
for rec in SeqIO.parse('Filtro_Proteínas_virales_DENV-2.fasta', 'fasta'):
    x = ProteinAnalysis(rec.seq)
    print(rec.id, '%0.2f' % x.instability_index())

prM_Proteína 41.70
E_Proteína 26.97
NS1_Proteína 47.46
NS2A_Proteína 30.27
NS2B_Proteína 43.16
NS3_Proteína 38.13
NS4A_Proteína 33.25
NS4B_Proteína 32.78
NS5_Proteína 36.44


In [12]:
## Índice alifático - aliphatic_index()
for rec in SeqIO.parse('Filtro_Proteínas_virales_DENV-2.fasta', 'fasta'):
    peptide = peptides.Peptide(rec.seq)
    print(rec.id, '%0.2f' % peptide.aliphatic_index())

prM_Proteína 74.64
E_Proteína 84.83
NS1_Proteína 72.56
NS2A_Proteína 125.69
NS2B_Proteína 117.77
NS3_Proteína 78.45
NS4A_Proteína 129.92
NS4B_Proteína 105.08
NS5_Proteína 72.11


In [13]:
## GRAVY - gravy()
for rec in SeqIO.parse('Filtro_Proteínas_virales_DENV-2.fasta', 'fasta'):
    x = ProteinAnalysis(rec.seq)
    print(rec.id, '%0.2f' % x.gravy())

prM_Proteína -0.22
E_Proteína -0.10
NS1_Proteína -0.50
NS2A_Proteína 0.68
NS2B_Proteína 0.31
NS3_Proteína -0.51
NS4A_Proteína 0.63
NS4B_Proteína 0.27
NS5_Proteína -0.61


Además, se evaluó la presencia de hélices transmembrana y algún péptido señal con un script en Python de los servidores en línea **SignalP v6.0** (https://services.healthtech.dtu.dk/services/SignalP-6.0/) (*Teufel et al., 2022*) y **DeepTMHMM** (https://dtu.biolib.com/DeepTMHMM) (*Hallgren et al., 2022*), respectivamente.

### - Evaluación de presencia de péptidos señal (SignalP v6.0)

In [24]:
# Descargar archivo tar.gz (https://services.healthtech.dtu.dk/services/SignalP-6.0/)
# Instalar el paquete descargado:
!pip install signalp-6-package/

Defaulting to user installation because normal site-packages is not writeable
Processing ./signalp-6-package
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for signalp6: filename=signalp6-6.0+g-py3-none-any.whl size=25106 sha256=e4550ebaf982c5df0b3428abf8bf1ebf2bab8121e7acbf324b69eacb4c422fc4
  Stored in directory: /home/victor/.cache/pip/wheels/c3/c5/f5/e6e08612c23af583c242633b144a6075f33bac8dda6605c11a
Successfully built signalp6
  Attempting uninstall: signalp6
    Found existing installation: signalp6 6.0+g
    Uninstalling signalp6-6.0+g:
      Successfully uninstalled signalp6-6.0+g


In [2]:
# Copiar los archivos del modelo en la ubicación en la que se instaló el módulo 'Signalp'
!SIGNALP_DIR = $(python3 -c 'import signalp; import os; print(os.path.dirname(signalp.__file__))' )
!cp -r signalp-6-package/models/* $SIGNALP_DIR signalp-6-package/signalp/model_weights/
## (En el caso de Jupyter, los archivos de 'sequential_models_signalp6' también deben moverse a '/home/victor/.local/lib/python3.10/site-packages/signalp/model_weights')

In [25]:
# Realizar la predicción de los péptidos señal
!signalp6 --fastafile Filtro_Proteínas_virales_DENV-2.fasta --organism other --output_dir SignalP_v6_Resultados --format txt --mode slow-sequential

Writing files: 100%|██████████████████████████████| 9/9 [00:00<00:00, 49.74it/s]


### - Evaluación de presencia de hélices transmembrana (DeepTMHMM)

In [47]:
# Instalar 'pybiolib'
!pip3 install -qU pybiolib
!pip3 install torch torchvision

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 1.5 MB/s eta 0:00:00m eta 0:00:010:00:01


In [26]:
# Importar 'pybiolib' y cargar 'DeepTMHMM'
import biolib
deeptmhmm = biolib.load('DTU/DeepTMHMM')

2023-05-28 12:53:15,464 | INFO : Loaded project DTU/DeepTMHMM:1.0.24


In [27]:
# Realizar la predicción de la presencia de hélices transmembrana
deeptmhmm_job = deeptmhmm.cli(args = '--fasta Filtro_Proteínas_virales_DENV-2.fasta')

2023-05-28 12:53:49,257 | INFO : Job "70361464-c143-464d-a331-8637161c148c" is starting...
2023-05-28 12:54:04,402 | INFO : Cloud: Initializing
2023-05-28 12:54:04,404 | INFO : Cloud: Pulling images...
2023-05-28 12:54:04,405 | INFO : Cloud: Computing...
Running DeepTMHMM on 9 sequences...
Step 1/4 | Loading transformer model...

Step 2/4 | Generating embeddings for sequences...
Generating embeddings: 100% 9/9 [00:01<00:00,  5.35seq/s]

Step 3/4 | Predicting topologies for sequences in batches of 1...
Topology prediction: 100% 9/9 [00:06<00:00,  1.43seq/s]

Step 4/4 | Generating output...
2023-05-28 12:54:28,574 | INFO : Cloud: Computation finished
2023-05-28 12:54:28,574 | INFO : Cloud: Result Ready


In [28]:
# Guardar los resultados
deeptmhmm_job.save_files('DeepTMHMM_Resultados')

2023-05-28 12:56:08,005 | INFO : Saving 3 files to DeepTMHMM_Resultados...
2023-05-28 12:56:09,876 | INFO :   - DeepTMHMM_Resultados/TMRs.gff3
2023-05-28 12:56:11,760 | INFO :   - DeepTMHMM_Resultados/predicted_topologies.3line
2023-05-28 12:56:13,637 | INFO :   - DeepTMHMM_Resultados/deeptmhmm_results.md


De igual manera, se estimó la localización subcelular y la tendencia a la solubilidad de las proteínas con los servidores **DeepLoc v2.0** (https://services.healthtech.dtu.dk/services/DeepLoc-2.0/) (*Thumuluri et al., 2022*) y **Protein-Sol** (https://protein-sol.manchester.ac.uk/) (*Hebditch et al., 2017*), respectivamente.

### - Estimación de localización subcelular (DeepLoc v2.0)

In [6]:
# Inicializar el Selenium WebDriver
driver = webdriver.Chrome(service = s)

# Visitar el servidor DeepLoc v2.0
driver.get('https://services.healthtech.dtu.dk/services/DeepLoc-2.0/')
time.sleep(1)
driver.find_element(by = By.XPATH, value = '//*[@id="cookiescript_accept"]').click()
time.sleep(4)

## Importar el archivo con las secuencias de las proteínas
input_proteins = driver.find_element(by = By.NAME, value = 'uploadfile')
input_proteins.send_keys('/home/victor/Escritorio/Tesis/RESULTADOS/3. Filtrado de proteínas virales objetivo/Filtro_Proteínas_virales_DENV-2.fasta')

# Entregar la data
elements = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/div[3]/div/div[2]/div[1]/div/form/div/div/div/div/input[1]')
driver.execute_script('arguments[0].click();', elements)
time.sleep(130)

# Obtener la predicción de localización subcelular
get_url = driver.current_url
time.sleep(4)
driver.find_element(by = By.XPATH, value = '/html/body/div[1]/div[2]/h5/a').click()


driver.close()

print('All sequences were downloaded!')

All sequences were downloaded!


### - Estimación de tendencia a solubilidad (Protein-Sol)

In [52]:
# Diseñar un archivo de entrada exclusivamente con las secuencias de las 9 proteínas filtradas
file_seq_filtered = open('Filtro_Proteínas_virales_DENV-2.fasta')

## Secuencias aminoacídicas de cada proteína
data_seq_filtered = []
for pos_f, l_num_f in enumerate(file_seq_filtered):
    # comprobar si el número de línea se especifica en las líneas para leer la matriz
    if pos_f %3 == 1:
        # imprimir el número de línea requerido
        data_seq_filtered.append(l_num_f)

each_seq_filtered = [[x.rstrip('\n')] for x in data_seq_filtered]

In [53]:
# Inicializar el Selenium WebDriver
driver = webdriver.Chrome(service = s)

# Definir los parámetros
## Website del servidor Protein-Sol
driver.get('https://protein-sol.manchester.ac.uk/')

# Esperar un segundo, hasta que la página esté totalmente cargada
time.sleep(1)

for data in each_seq_filtered:
    count = 0
    
    ## Nombre de la proteína objetivo 
    protein = driver.find_element(by = By.NAME, value = 'sequence-input')

    ## Secuencia de la proteína objetivo
    sequence = driver.find_elements(by = By.NAME, value = 'sequence-input')

    ## Entrega de la data
    submitButton = driver.find_element(by = By.NAME, value = 'singleprediction')

    # Estimar la tendencia a solubilidad:
    for value in sequence:
        value.send_keys(data[count])
        count += 1
    submitButton.click()
    # Descargar el resultado
    driver.find_element(by = By.ID, value = 'predictionButton').click()
    driver.find_element(by = By.CLASS_NAME, value = 'headerlogo').click()
    
driver.close()

print('All the sequences were processed and their results were downloaded successfully!')

All the sequences were processed and their results were downloaded successfully!
